# 4D numerical experiment of the inverted pendulum

In [2]:
import gpytorch
import torch
from src.TVBO import TimeVaryingBOModel
from src.objective_functions_LQR import lqr_objective_function_4D


### Hyperparameters

In [3]:
# parameters regarding the objective function
objective_function_options = {'objective_function': lqr_objective_function_4D,

                              # optimize the 4D feedback gain
                              'spatio_dimensions': 4,

                              # approximate noise level from the objective function
                              'noise_lvl': 0.005,

                              # feasible set for the optimization
                              'feasible_set': torch.tensor([[-3.5, -7, -62.5, -5],
                                                            [-1.5, -4, -12.5, -1]],
                                                           dtype=torch.float),

                              # initial feasible set consisting of only controllers
                              'initial_feasible_set': torch.tensor([[-3, -6, -50, -4],
                                                                    [-2, -4, -25, -2]],
                                                                   dtype=torch.float),

                              # scaling \theta to have approximately equal lengthscales in each dimension
                              'scaling_factors': torch.tensor([1 / 8, 1 / 4, 3, 1 / 4])}

# parameters regarding the model
model_options = {'constrained_dimensions': None,  # later specified for each variation
                 'forgetting_type': None,  # later specified for each variation
                 'forgetting_factor': None,  # later specified for each variation

                 # specification for the constraints  (cf. Agrell 2019)
                 'nr_samples': 10000,
                 'xv_points_per_dim': 4,  # VOPs per dimensions
                 'truncation_bounds': [0, 2],

                 # specification of prior
                 'prior_mean': 0.,
                 'lengthscale_constraint': gpytorch.constraints.Interval(0.5, 6),
                 'lengthscale_hyperprior': gpytorch.priors.GammaPrior(6, 1 / 0.3),
                 'outputscale_constraint_spatio': gpytorch.constraints.Interval(0, 20),
                 'outputscale_hyperprior_spatio': None,
                 'n_initial_points': 5 }

### Specify variations

In [4]:
# UI -> UI-TVBO, B2P_OU -> TV-GP-UCB
variations = [
    # in the paper color blue
    {'forgetting_type': 'UI', 'forgetting_factor': 0.03, 'constrained_dims': [], 'prior_mean': []},
    
    # OUR APPROACH
    {'forgetting_type': 'UI', 'forgetting_factor': 0.03, 'constrained_dims': [], 'prior_mean': 'DynaBO'},


    # in the paper color red
    {'forgetting_type': 'B2P_OU', 'forgetting_factor': 0.03, 'constrained_dims': [], 'prior_mean': []}, ]

### Start optimization

In [11]:
trials_per_variation = 25  # number of different runs
for variation in variations:

    # update variation specific parameters
    model_options['forgetting_type'] = variation['forgetting_type']
    model_options['forgetting_factor'] = variation['forgetting_factor']
    model_options['constrained_dimensions'] = variation['constrained_dims']
    n_initial_points = model_options['n_initial_points']
    
    # specify name to safe results
    method_name = model_options['forgetting_type']
    forgetting_factor = model_options['forgetting_factor']
    string = 'constrained' if model_options['constrained_dimensions'] else 'unconstrained'

    NAME = f"{method_name}_4DLQR_{string}_forgetting_factor_{forgetting_factor}_n_initial_points_{n_initial_points}".replace('.', '_')

    if variation['prior_mean'] == 'DynaBO':
        model_options['prior_mean'] = variation['prior_mean']
        NAME = "DynaBOn_" + NAME

    tvbo_model = TimeVaryingBOModel(objective_function_options=objective_function_options,
                                    model_options=model_options,
                                    post_processing_options={},
                                    add_noise=False, )  # noise is added during the simulation of the pendulum

    # run optimization
    for trial in range(1, trials_per_variation + 1):
        # tvbo_model.run_TVBO(n_initial_points=30,
        #                     time_horizon=300,
        #                     safe_name=NAME,
        #                     trial=trial, )

        tvbo_model.run_TVBO(n_initial_points=n_initial_points,
                    time_horizon=300,
                    safe_name=NAME,
                    trial=trial, )
    print('Finished.')

Initialization


/zfsauton2/home/acapone2/anaconda3/envs/uitvbo/lib/python3.9/site-packages/botorch/fit.py:148: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  warnings.warn(w.message, w.category)


Time for optimizing aquisition functions: 0.393s.

Timestep    5 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Parameter name: lengthscale 0 value = 1.495
Parameter name: lengthscale 1 value = 1.496
Parameter name: lengthscale 2 value = 1.474
Parameter name: lengthscale 3 value = 1.499
Time for optimizing aquisition functions: 0.263s.
Time for optimizing aquisition functions: 0.684s.
Time taken for timestep    5: 1.136s.

Timestep    6 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Parameter name: lengthscale 0 value = 1.577
Parameter name: lengthscale 1 value = 1.557
Parameter name: lengthscale 2 value = 1.610
Parameter name: lengthscale 3 value = 1.505
Time for optimizing aquisition functions: 0.272s.
Time for optimizing aquisition functions: 0.366s.
Time taken for timestep    6: 0.743s.

Timestep    7 of Trail  1.
Parameter name: noise value = 0.000
Parameter name: outputscale value = 1.000
Para

KeyboardInterrupt: 

In [ ]:
import os
import pickle
path = './results/'
fxs_ours = []
fxs = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'results_DynaBOn_UI_4DLQR_unconstrained' in i:
        with open('./results/' + i, 'rb') as handle:
            results = pickle.load(handle)
            fxs_ours.append(torch.tensor(results['f_of_x'])*results['inital_data_mean_and_stdv'][1] + results['inital_data_mean_and_stdv'][0])
    if os.path.isfile(os.path.join(path,i)) and 'results_UI_4DLQR_unconstrained' in i:
        with open('./results/' + i, 'rb') as handle:
            results = pickle.load(handle)
            fxs.append(torch.tensor(results['f_of_x'])*results['inital_data_mean_and_stdv'][1] + results['inital_data_mean_and_stdv'][0])
min_shape = min([fx.shape[0] for fx in fxs + fxs_ours])
regret = torch.hstack([fx[:min_shape].sum() for fx in fxs]).mean()
regret_ours = torch.hstack([fx[:min_shape].sum() for fx in fxs_ours]).mean()